<center><h1>Least Squares Support Vector Classifier</h1></center>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/LSSVM-master')


## Summary:
1. [Introduction](#introduction)


2. [Using the classifier](#using_classifier)

    2.1 [CPU/Numpy version](#cpu_version)
    
    2.2 [GPU/PyTorch version](#gpu_version)

# 1. Introduction <a class="anchor" id="introduction"></a>

The Least Squares Support Vector Machine (LSSVM) is a variation of the original Support Vector Machine (SVM) in which we have a slight change in the objective and restriction functions that results in a big simplification of the optimization problem.

First, let's see the optimization problem of an SVM:

$$
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + C \sum_{i=1}^{n} \xi_i &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b)\geq 1 - \xi_i, && i = 1,..., n \\
         && \xi_i \geq 0,                            && i = 1,..., n
\end{align}
$$

In this case, we have a set of inequality restrictions and when solving the optimization problem by it's dual we find a discriminative function, adding the kernel trick, of the type:


$$ f(\vec{x}) = sign \ \Big( \sum_{i=1}^{n} \alpha_i^o y_i K(\vec{x}_i,\vec{x}) + b_o \Big) $$

Where $\alpha_i^o$ and $b_o$ denote optimum values. Giving enough regularization (smaller values of $C$) we get a lot of $\alpha_i^o$ nulls, resulting in a sparse model in which we only need to save the pairs $(\vec{x}_i,y_i)$ which have the optimum dual variable not null. The vectors $\vec{x}_i$ with not null $\alpha_i^o$ are known as support vectors (SV).



In the LSSVM case, we change the inequality restrictions to equality restrictions. As the $\xi_i$ may be negative we square its values in the objective function:

$$
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + \gamma \frac{1}{2}\sum_{i=1}^{n} \xi_i^2 &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b) = 1 - \xi_i, && i = 1,..., n
\end{align}
$$


The dual of this optimization problem results in a system of linear equations, a set of Karush-Khun-Tucker (KKT) equations:

$$
\begin{bmatrix}
    0 & \vec{d}^T \\
    \vec{y} & \Omega + \gamma^{-1} I
\end{bmatrix}
\
\begin{bmatrix}
    b  \\
    \vec{\alpha}
\end{bmatrix}
=
\begin{bmatrix}
    0 \\
    \vec{1}
\end{bmatrix}
$$

Where, with the kernel trick, &nbsp; $\Omega_{i,j} = y_i y_j K(\vec{x}_i,\vec{x}_j)$,  &nbsp;  $\vec{y} = [y_1 \ y_2 \ ... \ y_n]^T$, &nbsp; $\vec{\alpha} = [\alpha_1 \ \alpha_2 \ ... \ \alpha_n]^T$ &nbsp;  e &nbsp; $\vec{1} = [1 \ 1 \ ... \ 1]^T$.

The discriminative function of the LSSVM has the same form of the SVM but the $\alpha_i^o$ aren't usually null, resulting in a bigger model. The big advantage of the LSSVM is in finding it's parameters, which is reduced to solving the linear system of the type:

$$ A\vec{z} = \vec{b} $$

A well-known solution of the linear system is when we minimize the square of the residues, that can be written as the optimization problem:

$$
\begin{align}
    minimize && f_o(\vec{z})=\frac{1}{2}||A\vec{z} - \vec{b}||^2\\
\end{align}
$$

And have the analytical solution:

$$ \vec{z} = A^{\dagger} \vec{b} $$

Where $A^{\dagger}$ is the pseudo-inverse defined as:

$$ A^{\dagger} = (A^T A)^{-1} A^T$$

# 2. Using the classifier <a class="anchor" id="using_classifier"></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_digits, fetch_olivetti_faces,fetch_lfw_people
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.datasets import cifar10


from lssvm import LSSVC, LSSVC_GPU
from utils.encoding import dummie2multilabel

In [ ]:
# Preprocessing

# Import digits recognition dataset (from sklearn)
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2020)

# Scaling features (from sklearn)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_tr_norm = scaler.transform(X_train)
X_ts_norm = scaler.transform(X_test)

# Get information about input and outputs
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape:  {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape:  {y_test.shape}")
print(f"np.unique(y_train): {np.unique(y_train)}")
print(f"np.unique(y_test):  {np.unique(y_test)}")

X_train.shape: (1078, 64)
X_test.shape:  (719, 64)
y_train.shape: (1078,)
y_test.shape:  (719,)
np.unique(y_train): [0 1 2 3 4 5 6 7 8 9]
np.unique(y_test):  [0 1 2 3 4 5 6 7 8 9]


## 2.1 CPU/Numpy version <a class="anchor" id="cpu_version"></a>

In [ ]:
# Use the classifier with different kernels
print("Classic MNIST")
print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')


Classic MNIST
Gaussian kernel:
acc_test =  0.9680111265646731 

precision =  0.9691141911658914 

recall =  0.9680111265646731 

f1 =  0.9653951004459118 

Polynomial kernel:
acc_test =  0.9944367176634215 

precision =  0.9947184028450203 

recall =  0.9944367176634215 

f1 =  0.9945003025345661 

Linear kernel:
acc_test =  0.9791376912378303 

precision =  0.9803825707180638 

recall =  0.9791376912378303 

f1 =  0.9795457737105719 



In [ ]:
# Preprocessing

X, y = fetch_olivetti_faces(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2020)

# Scaling features (from sklearn)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_tr_norm = scaler.transform(X_train)
X_ts_norm = scaler.transform(X_test)

# Get information about input and outputs
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape:  {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape:  {y_test.shape}")
print(f"np.unique(y_train): {np.unique(y_train)}")
print(f"np.unique(y_test):  {np.unique(y_test)}")

X_train.shape: (240, 4096)
X_test.shape:  (160, 4096)
y_train.shape: (240,)
y_test.shape:  (160,)
np.unique(y_train): [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
np.unique(y_test):  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 33 34 35 36 37 38 39]


In [ ]:
# Use the classifier with different kernels
print("'Olivetti facces")
print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

'Olivetti facces
Gaussian kernel:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


acc_test =  0.98125
precision =  0.9628515625
recall =  0.98125
f1 =  0.9719637223974763 

Polynomial kernel:
acc_test =  1.0
precision =  1.0
recall =  1.0
f1 =  1.0 

Linear kernel:
acc_test =  1.0
precision =  1.0
recall =  1.0
f1 =  1.0 



In [ ]:
# Preprocessing

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Scale the features
scaler = MinMaxScaler()
scaler.fit(X_train.reshape(-1, 32*32*3))  # Reshape to apply scaling to individual pixel values
x_train_norm = scaler.transform(X_train.reshape(-1, 32*32*3))
x_test_norm = scaler.transform(X_test.reshape(-1, 32*32*3))

# Split the data into 20% for training
x_train_subset, x_val_subset, y_train_subset, y_val_subset = train_test_split(
    x_train_norm, y_train, test_size=0.8, random_state=42)

# # Further split the 10% for training and validation if needed
# x_train_subset, x_val_subset, y_train_subset, y_val_subset = train_test_split(
#     x_train_subset, y_train_subset, test_size=0.5, random_state=42)


scaler = MinMaxScaler()
scaler.fit(X_train.reshape(-1, 32*32*3))  # Reshape to apply scaling to individual pixel values
x_train_norm = scaler.transform(X_train.reshape(-1, 32*32*3))
x_test_norm = scaler.transform(X_test.reshape(-1, 32*32*3))


X_tr_norm = x_train_subset
X_ts_norm = x_val_subset
y_train = y_train_subset
y_test = y_val_subset
# Get information about input and outputs
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape:  {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape:  {y_test.shape}")
print(f"np.unique(y_train): {np.unique(y_train)}")
print(f"np.unique(y_test):  {np.unique(y_test)}")

X_train.shape: (50000, 32, 32, 3)
X_test.shape:  (10000, 32, 32, 3)
y_train.shape: (10000, 1)
y_test.shape:  (40000, 1)
np.unique(y_train): [0 1 2 3 4 5 6 7 8 9]
np.unique(y_test):  [0 1 2 3 4 5 6 7 8 9]


In [ ]:
# Use the classifier with different kernels
import time


print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation

start_time = time.time()
lssvc.fit(X_tr_norm, y_train) # Fitting the model
end_time = time.time()
training_time_seconds = end_time - start_time
training_time_formatted = time.strftime("%H:%M:%S", time.gmtime(training_time_seconds))
print(f"Total training time: {training_time_formatted}")

y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')










print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)

start_time = time.time()
lssvc.fit(X_tr_norm, y_train) # Fitting the model
end_time = time.time()
training_time_seconds = end_time - start_time
training_time_formatted = time.strftime("%H:%M:%S", time.gmtime(training_time_seconds))
print(f"Total training time: {training_time_formatted}")


y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')















print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')

start_time = time.time()
lssvc.fit(X_tr_norm, y_train) # Fitting the model
end_time = time.time()
training_time_seconds = end_time - start_time
training_time_formatted = time.strftime("%H:%M:%S", time.gmtime(training_time_seconds))
print(f"Total training time: {training_time_formatted}")


y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

Gaussian kernel:
Total  10
Label
0
Label
1
Label
2
Label
3
Label
4
Label
5
Label
6
Label
7
Label
8
Label
9
Total training time: 01:36:55


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


acc_test =  0.899725
precision =  0.809505075625
recall =  0.899725
f1 =  0.852233955572517 

Polynomial kernel:
Total  10
Label
0
Label
1
Label
2
Label
3
Label
4
Label
5
Label
6
Label
7
Label
8
Label
9
Total training time: 01:02:38
acc_test =  0.9099
precision =  0.8923756098986593
recall =  0.9099
f1 =  0.8930190850899955 

Linear kernel:
Total  10
Label
0
Label
1
Label
2
Label
3
Label
4
Label
5
Label
6
Label
7
Label
8
Label
9
Total training time: 00:59:38
acc_test =  0.86735
precision =  0.870146605085178
recall =  0.86735
f1 =  0.8687229347862653 



In [ ]:
 !pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Upload kaggle.json
# please follow this link incase not aware: https://www.kaggle.com/general/74235
from google.colab import files
files.upload()

! pip install opendatasets --upgrade
import opendatasets as od

dataset_url = 'https://www.kaggle.com/moltean/fruits'
od.download(dataset_url)

Saving kaggle.json to kaggle.json


100%|██████████| 1.28G/1.28G [00:54<00:00, 25.5MB/s]


In [ ]:
!ls fruits/fruits-360_dataset/fruits-360


LICENSE  papers  readme.md  Test  test-multiple_fruits	Training


In [ ]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import string
from mlxtend.plotting import plot_decision_regions
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from sklearn.svm import SVC

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch.utils.data import Dataset
import os
import PIL.Image as Image



# --- Step 1: Load the Fruits 360 Dataset ---
data_dir = 'fruits/fruits-360_dataset/fruits-360/'
dim = 100
def getYourFruits(fruits, data_type, print_n=False, k_fold=False):
    images = []
    labels = []
    val = ['Training', 'Test']
    if not k_fold:
        path = data_dir + data_type + "/"
        for i,f in enumerate(fruits):
            p = path + f
            j=0
            for image_path in glob.glob(os.path.join(p, "*.jpg")):
                image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                image = cv2.resize(image, (dim, dim))
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                images.append(image)
                labels.append(i)
                j+=1
            if(print_n):
                print("There are " , j , " " , data_type.upper(), " images of " , fruits[i].upper())
        images = np.array(images)
        labels = np.array(labels)
        return images, labels
    else:
        for v in val:
            path = data_dir + v + "/"
            for i,f in enumerate(fruits):
                p = path + f
                j=0
                for image_path in glob.glob(os.path.join(p, "*.jpg")):
                    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                    image = cv2.resize(image, (dim, dim))
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    images.append(image)
                    labels.append(i)
                    j+=1
        images = np.array(images)
        labels = np.array(labels)
        return images, labels

def getAllFruits():
    fruits = []
    for fruit_path in gl:
      ob.glob(data_dir + "Training/*")
      fruit = fruit_path.split("/")[-1]
      fruits.append(fruit)
    return fruits


selected_classes = ['Apple', 'Banana', 'Cherry']



In [ ]:
# Select Fruits
fruits = ['Banana' , 'Cherry 1'] #Binary classification

#Get Images and Labels
X_t, y_train =  getYourFruits(fruits, 'Training', print_n=True, k_fold=False)
X_test, y_test = getYourFruits(fruits, 'Test', print_n=True, k_fold=False)

#Get data for k-fold
X,y = getYourFruits(fruits, '', print_n=True, k_fold=True)

#Scale Data Images
scaler = StandardScaler()
X_tr_norm = scaler.fit_transform([i.flatten() for i in X_t])
X_ts_norm = scaler.fit_transform([i.flatten() for i in X_test])
X = scaler.fit_transform([i.flatten() for i in X])

# Get information about input and outputs
print(f"X_train.shape: {X_tr_norm.shape}")
print(f"X_test.shape:  {X_ts_norm.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape:  {y_test.shape}")
print(f"np.unique(y_train): {np.unique(y_train)}")
print(f"np.unique(y_test):  {np.unique(y_test)}")


There are  490   TRAINING  images of  BANANA
There are  492   TRAINING  images of  CHERRY 1
There are  166   TEST  images of  BANANA
There are  164   TEST  images of  CHERRY 1
X_train.shape: (982, 30000)
X_test.shape:  (330, 30000)
y_train.shape: (982,)
y_test.shape:  (330,)
np.unique(y_train): [0 1]
np.unique(y_test):  [0 1]


In [ ]:
# Use the classifier with different kernels
print("Fruits 360")
print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred))
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

Fruits 360
Gaussian kernel:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


acc_test =  0.49696969696969695
precision =  0.24697887970615243
recall =  0.49696969696969695
f1 =  0.3299717826033615 

Polynomial kernel:
acc_test =  0.9848484848484849
precision =  0.9852915116073012
recall =  0.9848484848484849
f1 =  0.9848436130234184 

Linear kernel:
acc_test =  1.0
precision =  1.0
recall =  1.0
f1 =  1.0 



In [ ]:
print("Fruits 360")
print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.1) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print("Fruits 360")
print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print("Fruits 360")
print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')

print("Fruits 360")
print('Gaussian kernel:')
lssvc = LSSVC(gamma=50, kernel='rbf', sigma=50) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making predictions with the trained model
acc = accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)) # Calculate Accuracy
precision = precision_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
recall = recall_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
f1 = f1_score(dummie2multilabel(y_test), dummie2multilabel(y_pred), average='weighted')
print('acc_test = ', acc)
print('precision = ', precision)
print('recall = ', recall)
print('f1 = ', f1, '\n')


Fruits 360
Gaussian kernel:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


acc_test =  0.49696969696969695
precision =  0.24697887970615243
recall =  0.49696969696969695
f1 =  0.3299717826033615 

Fruits 360
Gaussian kernel:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


acc_test =  0.49696969696969695
precision =  0.24697887970615243
recall =  0.49696969696969695
f1 =  0.3299717826033615 

Fruits 360
Gaussian kernel:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


acc_test =  0.49696969696969695
precision =  0.24697887970615243
recall =  0.49696969696969695
f1 =  0.3299717826033615 

Fruits 360
Gaussian kernel:
acc_test =  0.9121212121212121
precision =  0.9253257968283874
recall =  0.9121212121212121
f1 =  0.911485098089641 



The user may also save the model in JSON format:

In [ ]:
lssvc.dump('model')
loaded_model = LSSVC.load('model')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(loaded_model.predict(X_ts_norm))
    )
)

## 2.2 GPU/PyTorch version <a class="anchor" id="gpu_version"></a>    

It has the same functionalities and syntax of the CPU version, the difference is the use of PyTorch to run the operations on the GPU.

In [ ]:
# Use the classifier with different kernels

print('Gaussian kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='rbf', sigma=.5)
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Polynomial kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='poly', d=2)
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Linear kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='linear')
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

The user may also save the model in JSON format:

In [ ]:
lssvc_gpu.dump('model')
loaded_model = LSSVC_GPU.load('model')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(loaded_model.predict(X_ts_norm).cpu())
    )
)

The `model.json` is the same for the CPU and GPU version, giving the developer the possibility to train a model in GPU, dumping it in a .json, and loading in CPU version (the other way around is also possible).

In [ ]:
lssvc.dump('model_from_cpu')
lssvc_gpu = LSSVC_GPU.load('model_from_cpu')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)

In [ ]:
lssvc_gpu.dump('model_from_gpu')
lssvc = LSSVC.load('model_from_gpu')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test),
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)